<a href="https://colab.research.google.com/github/junfeiai/Info_CcGAN_OPF/blob/main/OPF_test_grid118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from matplotlib import pyplot as plt
import sys
import pandas as pd
from pandas import DataFrame as df
from numpy.random import randn
from scipy.io import loadmat
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model,Sequential,Model
import math
import time
from tensorflow.keras.layers import Conv1D, MaxPooling1D,Lambda,Concatenate
import pdb; 
import scipy
from keras.constraints import Constraint
import datetime
from scipy.stats import weibull_min

In [ ]:
mat_load = loadmat('IEEE118_load.mat')
opf_condition = loadmat('grid118_fc_new_test_conditions_by_opf.mat')['conditions_list']
opf_points = loadmat('grid118_fc_new_test_points_by_opf.mat')['datapoints_list']
mat_y = loadmat('Y_bus118.mat')
mat_gen = loadmat('IEEE118_gen.mat')['gen118']
mat_gencost = loadmat('case118_gencost.mat')['gencost']
Y_bus = mat_y['Y_bus118'].toarray().astype('complex64')
gen_ids=mat_gen[:,0]-1
baseMVA = 100
case118_pload = mat_load['aa'][:,2]/baseMVA
case118_qload = mat_load['aa'][:,3]/baseMVA
opf_real_conditions = opf_condition[:]/baseMVA
opf_real_vm= opf_points[:,0:118]
opf_real_va = opf_points[:,118:118*2]
opf_matpower_pq = opf_points[:,118*2:]
opf_matpower_p = opf_points[:,236:290]
opf_matpower_q = opf_points[:,290:]
opf_real_p = np.zeros([opf_points.shape[0],118])
opf_real_q = np.zeros([opf_points.shape[0],118])
j=0
for i in range(0,118):
  if i in gen_ids:
    opf_real_p[:,i]=opf_matpower_p[:,j]
    opf_real_q[:,i]=opf_matpower_q[:,j]
    j=j+1
opf_real_p=opf_real_p/baseMVA
opf_real_q=opf_real_q/baseMVA

gen_id=mat_gen[:,0]-1
non_gen_ids = np.zeros([64])
j=0
for i in range(0,118):
  if i not in gen_id:
    non_gen_ids[j]=i
    j=j+1
wind_der_ids = non_gen_ids[:-40]
solar_der_ids = non_gen_ids[-40:]

basis_vector = np.zeros(118)
for i in range(0,118):
  if i in gen_ids:
    basis_vector[i]=1

In [ ]:
def get_gencost(number_of_buses,mat_gen,mat_gencost):
  """
  Function to format financial cost efficients in MatPower 
  to a tensor.
  :number_of_buses: determins the dimension of the tensor
  :mat_gen: generator list in MatPower
  :mat_gencost: generator cost in MatPower
  :return: tf_c1,tf_c2 are the first and second order coefficient tensors
  """ 
  gen_ids = mat_gen[:,0]-1
  c1=np.zeros(number_of_buses)
  c2=np.zeros(number_of_buses)
  j=0
  for i in range(0,number_of_buses):
    if i in gen_ids:
      c1[i] = mat_gencost[j,5]
      c2[i] = mat_gencost[j,4]
      j=j+1
  tf_c1 = tf.convert_to_tensor(c1,dtype='float32')
  tf_c2 = tf.convert_to_tensor(c2,dtype='float32')
  return tf_c1,tf_c2

# use the generator to generate n fake examples, with class labels
def generate_test_samples(generator, latent_dim, test_samples):
  # generate points in latent space
  n_samples = opf_real_conditions.shape[0]
  z_input, labels_input = generate_latent_points(latent_dim, n_samples)
  # predict outputs
  images = generator.predict([z_input, opf_real_conditions])
  # create class labels
  y = np.ones((n_samples, 1))
  return opf_real_conditions,images, y

def optimality_checking(p_opt,q_opt,p_fake,q_fake):
  #pdb.set_trace()
  p_mse = K.sum(K.square(p_opt*100-p_fake*100))
  q_mse = K.sum(K.square(q_opt*100-q_fake*100))
  return p_mse,q_mse

def financial_optimality_checking(p_opt,q_opt,p_fake,q_fake):
  #pdb.set_trace()
  p_opt_actual = tf.convert_to_tensor(p_opt*100,dtype=tf.float32)
  p_fake_actual = tf.convert_to_tensor(p_fake*100,dtype=tf.float32)
  #pdb.set_trace()
  p_opt_actual_cost = K.sum(tf.math.multiply(p_opt_actual, tf_c1)+\
                              tf.math.multiply(K.square(p_opt_actual), tf_c2))
  p_fake_actual_cost = K.sum(tf.math.multiply(p_fake_actual, tf_c1)+\
                            tf.math.multiply(K.square(p_fake_actual), tf_c2))
  p_mse = K.sum(p_opt_actual_cost-p_fake_actual_cost)
  print(p_opt_actual_cost)
  print(p_fake_actual_cost)
  print(p_mse)
  return p_mse


def generate_latent_points(latent_dim, n_samples):
  # generate points in the latent space
  code_input = np.random.uniform(-1,-0.6,n_samples).reshape([n_samples,1])
  x_input = randn(latent_dim * n_samples)
  z_input = x_input.reshape(n_samples, latent_dim)
  # generate labels
  p_demand,q_demand = rdm_load_pq_20per(case118_pload,case118_qload,n_samples,wind_der_ids,solar_der_ids)
  labels = np.concatenate([p_demand,q_demand],axis=1)
  return [code_input,z_input, labels]

def rdm_load_pq_20per(case_pload,case_qload,sample_number,wind_der_ids,solar_der_ids):
    #pdb.set_trace()
    wind_sum=0
    solar_sum=0
    number_of_bus = case_pload.shape[0]
    p_demand = np.zeros([sample_number,number_of_bus])
    q_demand = np.zeros([sample_number,number_of_bus])
    for i in range(0,number_of_bus):
        if case_pload[i]!=0:
          pi = case_pload[i]
          qi = case_qload[i]
          p_mw = np.random.uniform(pi*0.8,pi*1.2,sample_number)
          q_mvar = np.random.uniform(qi*0.8,qi*1.2,sample_number)
          p_demand[:,i] = p_mw
          q_demand[:,i] = q_mvar
          der_i=0
        if i in wind_der_ids:
          #for each wind farm, there are 6 wind turbines
          der_i = draw_wind_generation(1,2,5,sample_number)
          wind_sum=wind_sum+np.mean(der_i)
        elif i in solar_der_ids:
          #for each PV generator, there are 3000 panels
          der_i = draw_PV_generation(10000,2,5,sample_number)
          solar_sum=solar_sum+np.mean(der_i)
        p_demand[:,i] = p_demand[:,i]-der_i
    #print(wind_sum)
    #print(solar_sum)
    return p_demand,q_demand

def rdm_gen_pv(p_min,p_max,sample_number):
  #pdb.set_trace()
  number_of_gen = p_min.shape[0]
  p_supply = np.zeros([sample_number,number_of_gen])
  for i in range(0,number_of_gen):
    #if mat_gen['gen118'][i,1]!=0:
    p_gen = np.random.uniform(p_min[i],p_max[i],sample_number)
    p_supply[:,i] = p_gen
  ###
  hard_clip_min = 0.96
  hard_clip_max = 1.06
  _mean=1
  _std=0.01
  a = (hard_clip_min - _mean) / _std
  b = (hard_clip_max - _mean) / _std

  result = truncnorm.rvs(a=a, b=b, loc=_mean,size=sample_number*(number_of_gen-1),scale=_std)
  ###
  vm_gen = result.reshape([sample_number,number_of_gen-1])
  #np.random.uniform(0.94,1.06,[sample_number,number_of_gen-1])
  return p_supply,vm_gen

def rdm_gen_pv_low(p_min,p_max,sample_number,mat_gencost):
  #pdb.set_trace()
  number_of_gen = p_min.shape[0]
  p_supply = np.zeros([sample_number,number_of_gen])
  for i in range(0,number_of_gen):
    if mat_gencost[i,5]==20 and mat_gencost[i,4]<=0.1:
      p_gen = np.random.uniform(p_min[i],p_max[i],sample_number)
      p_supply[:,i] = p_gen
  vm_gen = np.random.uniform(0.94,1.06,[sample_number,number_of_gen-1])
  return p_supply,vm_gen

def draw_wind_generation(number_of_turbines, k,c,sample_number):
  x = weibull_min.rvs(k, loc=0, scale=c, size=sample_number)
  p=0.5*1.25*np.power(x,3)*1800/1000000
  return p*number_of_turbines

def draw_PV_generation(number_of_panels,k,c,sample_number):
  x = weibull_min.rvs(k, loc=0, scale=c, size=sample_number)
  return x*number_of_panels/1000000

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
  # make weights in the discriminator not trainable
  d_model.trainable = False
  [g_active,g_reactive,bus_vm,bus_va] = generator.output
  [gen_code, gen_noise, gen_label] = generator.input
  gan_output = d_model([tf.reshape(gen_label,[-1,1,118*2]),tf.reshape(g_active,[-1,1,118]),\
                        tf.reshape(g_reactive,[-1,1,118]),tf.reshape(bus_vm,[-1,1,118]),\
                        tf.reshape(bus_va,[-1,1,118])])
  # define gan model as taking noise and label and outputting a classification
  model = Model([gen_code, gen_noise, gen_label], gan_output)
  return model

def feasibility_checking(p_demand,q_demand, active_p,reactive_q,vm,va):
  #pdb.set_trace()
  generated_size = p_demand.shape[0]
  #Step1: calculate power withdraw on each bus
  P_out = tf.convert_to_tensor(p_demand-active_p,dtype='float32')
  Q_out = tf.convert_to_tensor(q_demand-reactive_q,dtype='float32')
  #pdb.set_trace()
  #get voltage on each bus
  v_r = tf.math.multiply(vm,tf.cos(tf.math.multiply(va,tf.constant(math.pi/180,dtype='float32'))))
  v_i = tf.math.multiply(vm,tf.sin(tf.math.multiply(va,tf.constant(math.pi/180,dtype='float32'))))
  V = tf.reshape(tf.complex(v_r,v_i),[generated_size,118])

  #calculate current
  Y_bus_tf = tf.convert_to_tensor(Y_bus)
  I = tf.matmul(V,Y_bus_tf)

  #calculate power injection on each bus
  S_in = tf.math.multiply(V,tf.math.conj(I))
  P_in = tf.math.real(S_in)
  Q_in = tf.math.imag(S_in)

  #evaluate the balance
  p_balance = tf.reduce_mean(P_in+P_out,axis=0)
  q_balance = tf.reduce_mean(Q_in+Q_out,axis=0)
  return p_balance,q_balance

def get_variation_cost(deltaP,mat_gen,mat_gencost):
  p_gr = mat_gen[:,8]
  c1 = mat_gencost[:,5]
  c2 = mat_gencost[:,4]
  sr=100
  rui = (3/60)/(p_gr/sr)
  delta_pi = (sr/rui)/(np.sum(sr/rui))*deltaP
  cost = (delta_pi*100)*c1+np.square((delta_pi*100))*c2
  return np.sum(cost)

def get_pq_bound(number_of_buses, mat_gen):
  number_of_gens = mat_gen.shape[0]
  p_upper = np.zeros(number_of_buses)
  q_upper = np.zeros(number_of_buses)
  q_lower = np.zeros(number_of_buses)
  j=0
  for i in range(0,number_of_buses):
    if i in (mat_gen[:,0]-1):
      #pdb.set_trace()
      p_upper[i]=mat_gen[j,8]
      q_upper[i]=mat_gen[j,3]
      q_lower[i]=mat_gen[j,4]
      j=j+1
  tf_p_upper = tf.convert_to_tensor(p_upper/baseMVA,dtype='float32')
  tf_q_upper = tf.convert_to_tensor(q_upper/baseMVA,dtype='float32')
  tf_q_lower = tf.convert_to_tensor(q_lower/baseMVA,dtype='float32')
  return tf_p_upper,tf_q_upper,tf_q_lower

def replacenan(t):
    return tf.where(tf.math.is_nan(t), tf.zeros_like(t), t)

In [ ]:
latent_dim = 200
discriminator = load_model('opf_gan/d.h5')
generator = load_model('opf_gan/g.h5')

In [ ]:
number=0
x = np.arange(-1,1,0.01)
z = np.zeros([200])
c = np.zeros([200])
tf_c1,tf_c2=get_gencost(118,mat_gen,mat_gencost)
c_input, z_input, labels_input = generate_latent_points(latent_dim, 1)
for i in range(0,200):
    c_input[0,0]=x[i]
    images = generator.predict([c_input.reshape([-1,1,1]),z_input.reshape([-1,1,latent_dim]),\
                                opf_real_conditions[number,:].reshape([1,1,236])])
    op=financial_optimality_checking(opf_real_p[number,:],opf_real_q[number,:],images[0],images[1])
    z[i]=op
    cost1=tf.math.multiply(images[0]*100, tf_c1)
    cost2=tf.math.multiply(K.square(images[0]*100), tf_c2)
    print(1)
    print(tf.reduce_sum(cost2))
    suma = tf.reduce_sum(cost1+cost2,axis=1)
    c[i]=suma
    print(str(i))
plt.plot(c[:])
plt.xlabel('Value of the Latent Code')
plt.ylabel('Cost Value')
plt.xticks(np.arange(0, 200, 20), np.arange(-10, 10, 2)/10)

In [ ]:
#Batch generation with Zero Order Optimization
tf_p_upper,tf_q_upper,tf_q_lower = get_pq_bound(118, mat_gen)
loss_acc = 0
loss_acc_list=[]
loss_per_list=[]
sdp_list = []
gan_list = []
feasibility_store_list=[]
actual_cost = []
gan_cost = []
demand_list = []
solutionp_list = []
solutionv_list = []
#Sample numeber
sample_batch = 50 #100,200,500,1000,5000
#Cost coefficients
tf_c1,tf_c2=get_gencost(118,mat_gen,mat_gencost)
#for 1000 test cases
for test_iter in range(0,1000):
  p_opt_actual = tf.convert_to_tensor(opf_real_p[test_iter,:]*100,dtype=tf.float32)
  cost1=tf.math.multiply(p_opt_actual, tf_c1)
  cost2=tf.math.multiply(K.square(p_opt_actual), tf_c2)
  suma = tf.reduce_sum(cost1+cost2)
  k=0
  number=test_iter
  #Generate latent points
  c_input, z_input, _ = generate_latent_points(latent_dim, sample_batch)
  tf_z=tf.convert_to_tensor(z_input.reshape([sample_batch,1,latent_dim]))
  images = generator.predict([c_input.reshape([-1,1,1]),tf_z,np.repeat(opf_real_conditions[number,:].reshape([-1,1,236]),sample_batch,axis=0)])
  y_a= K.sum(tf.math.multiply(images[0]*100, tf_c1)+tf.math.multiply(K.square(images[0]*100), tf_c2),axis=1)
  solutions = images[0]
  _,score,score2 = discriminator([np.repeat(opf_real_conditions[number,0:118*2].reshape([-1,1,118*2]),sample_batch,axis=0),\
                                          tf.reshape(images[0],[-1,1,118]),\
                                          tf.reshape(images[1],[-1,1,118]),\
                                          tf.reshape(images[2],[-1,1,118]),\
                                          tf.reshape(images[3],[-1,1,118])])
  candidates =y_a.numpy()
  best_index = np.argmin(score+score2*0.5)
  best_score1 = score[best_index]
  best_score2 = score2[best_index]
  P = images[0]
  tf_p = images[0]
  tf_q = images[1]
  tf_v = images[2]
  tf_phi = images[3]
  best_p = tf.reshape(tf_p[best_index],[-1,1,118])
  best_q = tf.reshape(tf_q[best_index],[-1,1,118])
  best_v = tf.reshape(tf_v[best_index],[-1,1,118])
  best_phi = tf.reshape(tf_phi[best_index],[-1,1,118])
  dual_v = np.zeros(1)
  dual_beta = np.zeros([1,1,118])
  dual_gamma = np.zeros([1,1,118])
  dual_epsilon = np.zeros(1)
  g_sign = np.zeros([1,1,118])
  p_gradient = np.zeros([1,1,118])
  update_dim = np.ones([1,1,118])
  for l in range(0,5):
    h=0.00000001
    best_p = best_p-0.000000001*p_gradient*update_dim
    best_p_plus = np.repeat(tf.reshape(best_p,[-1,1,118]),118,axis=0)+(np.identity(118)*basis_vector).reshape([-1,1,118])*h
    best_p_minus = np.repeat(tf.reshape(best_p,[-1,1,118]),118,axis=0)-(np.identity(118)*basis_vector).reshape([-1,1,118])*h
    _,score_plus,score_plus2 = discriminator([np.repeat(opf_real_conditions[number,0:118*2].reshape([-1,1,118*2]),118,axis=0),\
                            best_p_plus,np.repeat(tf.reshape(best_q,[-1,1,118]),118,axis=0),\
                            np.repeat(tf.reshape(best_v,[-1,1,118]),118,axis=0),\
                            np.repeat(tf.reshape(best_phi,[-1,1,118]),118,axis=0)])
    _,score_minus,score_minus2 = discriminator([np.repeat(opf_real_conditions[number,0:118*2].reshape([-1,1,118*2]),118,axis=0),\
                            best_p_minus,np.repeat(tf.reshape(best_q,[-1,1,118]),118,axis=0),\
                            np.repeat(tf.reshape(best_v,[-1,1,118]),118,axis=0),\
                            np.repeat(tf.reshape(best_phi,[-1,1,118]),118,axis=0)])
    dj = tf.reshape((score_plus-score_minus),[1, 1, 118])/(h*2)
    dj2 = tf.reshape((score_plus2-score_minus2),[1, 1, 118])/(h*2)
    p_gradient = 20000*tf.math.multiply(best_p, tf_c2)+tf_c1*100+dual_epsilon*dj2+dual_v*dj+dual_beta-dual_gamma
    if l==0:
      g_sign = np.sign(p_gradient)
      np.place(g_sign, g_sign==0., 2)
    if np.all(np.sign(p_gradient)!=g_sign):
      break
    else:
      update_dim = np.sign(p_gradient)==g_sign
      print(np.sum(update_dim))
    best_p = tf.math.minimum(tf.math.maximum(best_p,tf.zeros([1,1,118])),tf.reshape(tf_p_upper,[1,1,118]))
    lagrangian = tf.reduce_sum(tf.math.multiply(best_p*100, tf_c1)+tf.math.multiply(K.square(best_p*100), tf_c2)+\
        dual_v*best_score1+dual_beta*(best_p-tf_p_upper)-dual_gamma*best_p+dual_epsilon*best_score2)
    dual_v = dual_v+lagrangian/best_score1
    dual_epsilon = dual_v+lagrangian/best_score2
    dual_beta = tf.nn.relu(dual_beta+replaceinf(replacenan(lagrangian/(best_p-tf_p_upper))))
    dual_gamma = tf.nn.relu(dual_gamma+replaceinf(replacenan(lagrangian/(-best_p))))
  cost1=tf.math.multiply(best_p*100, tf_c1)
  cost2=tf.math.multiply(K.square(best_p*100), tf_c2)
  suma1 = tf.reduce_sum(cost1+cost2)
  end_time = datetime.datetime.now()
  y_b=suma1
  err = y_b-suma
  err_rate = err/suma
  loss_acc_list.append(err)
  loss_per_list.append(err_rate)
  sdp_list.append(suma)
  gan_list.append(y_b)
  _,aa,_=discriminator([opf_real_conditions[number,0:118*2].reshape([-1,1,118*2]),best_p,best_q,best_v,best_phi])
  feasibility_store_list.append(aa)
  demand_list.append(opf_real_conditions[number,:])
  solutionp_list.append(images[0][np.argmin(score)])
  solutionv_list.append(images[2][np.argmin(score)])

In [ ]:
mean_value = np.mean(np.absolute(loss_per_list))
deviation = np.sqrt(np.sum(np.square(loss_per_list-mean_value))/len(loss_per_list)) 
print(np.mean(np.absolute(loss_per_list)))
print(deviation*1.96/np.sqrt(len(loss_per_list)))